In [129]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [130]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [131]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [132]:
input_data = df_to_tensor(spectrum_train)
output_data = df_to_tensor(temp_train)

In [133]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [134]:
input_size = 66
output_size = 11
model = Net(input_size=input_size, output_size=output_size)

In [135]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [136]:
num_epochs = 100
batch_size = 32
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 10/100, Loss: 2078.165039
Epoch 20/100, Loss: 2061.452148
Epoch 30/100, Loss: 2039.547241
Epoch 40/100, Loss: 2006.534058
Epoch 50/100, Loss: 1963.285889
Epoch 60/100, Loss: 1913.534424
Epoch 70/100, Loss: 1875.365601
Epoch 80/100, Loss: 1890.545044
Epoch 90/100, Loss: 1925.291748
Epoch 100/100, Loss: 1865.003906


In [137]:
predictions = model(df_to_tensor(spectrum_test))

In [138]:
pd.DataFrame(predictions.detach().numpy(), columns=temp_test.columns).head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
0,528.749023,528.752686,531.136658,532.036316,533.051453,535.113708,535.755737,535.902588,534.847168,534.207947,533.267883
1,528.749023,528.752686,531.136658,532.036316,533.051453,535.113708,535.755737,535.902588,534.847168,534.207947,533.267883
2,528.749023,528.752686,531.136658,532.036316,533.051453,535.113708,535.755737,535.902588,534.847168,534.207947,533.267883
3,528.749023,528.752686,531.136658,532.036316,533.051453,535.113708,535.755737,535.902588,534.847168,534.207947,533.267883
4,528.749023,528.752686,531.136658,532.036316,533.051453,535.113708,535.755737,535.902588,534.847168,534.207947,533.267883


In [139]:
temp_test.head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,layer 11
0,410.761201,412.678794,409.482872,401.948393,398.928932,397.948393,401.340736,408.678794,416.761201,421.621765,421.254224
1,408.690133,411.339048,412.411804,410.608647,411.000000,407.948393,403.411804,400.018540,399.690133,402.961511,409.183156
2,414.296735,417.678794,413.018406,401.948393,395.393398,392.948393,397.805202,408.678794,420.296735,426.621765,424.789758
3,411.190133,415.669175,417.411804,414.938774,413.500000,407.948393,400.911804,395.688413,394.690133,398.631384,406.683156
4,415.482872,416.788008,412.019461,402.726045,398.928932,398.726045,403.877325,412.788008,421.482872,425.993518,424.749862
